In [ ]:
memory.default.employees

In [ ]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Validation query with ALL ROWS PER MATCH
query = """
    SELECT * FROM memory.default.employees MATCH_RECOGNIZE (
        PARTITION BY department, region
        ORDER BY hire_date
        MEASURES 
            salary AS current_salary,
            RUNNING SUM(salary) AS running_sum,
            MATCH_NUMBER() AS match_num
        ALL ROWS PER MATCH
        PATTERN (A+)
        DEFINE A AS salary > 1000
    );
    """
    
data = [
    {"id": 1, "name": "Alice",   "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob",     "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana",   "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
]
    
output_df = match_recognize(query, pd.DataFrame(data))
print("Match Recognize Output:")
print(output_df)


In [ ]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Validation query with ALL ROWS PER MATCH
query = """
    SELECT * FROM memory.default.employees MATCH_RECOGNIZE (
        PARTITION BY department, region
        ORDER BY hire_date
        MEASURES 
            salary AS current_salary,
            RUNNING SUM(salary) AS running_sum,
            MATCH_NUMBER() AS match_num
        ALL ROWS PER MATCH
        PATTERN (A*)
        DEFINE A AS salary > 1000
    );
    """
    
data = [
    {"id": 1, "name": "Alice",   "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob",     "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana",   "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
]
    
output_df = match_recognize(query, pd.DataFrame(data))
print("Match Recognize Output:")
print(output_df)


In [ ]:
import pandas as pd
from src.executor.match_recognize import match_recognize
# Use an absolute import for match_recognize.

query = """
    SELECT * FROM memory.default.employees MATCH_RECOGNIZE (
        PARTITION BY department, region
        ORDER BY hire_date
        MEASURES salary AS avg_salary
        PATTERN (A+)
        DEFINE A AS salary > 1000
    );
    """
    
data = [
        {"id": 1, "name": "Alice",   "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
        {"id": 2, "name": "Bob",     "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
        {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
        {"id": 4, "name": "Diana",   "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
    ]
    
output_df = match_recognize(query, pd.DataFrame(data))
print("Match Recognize Output:")
print(output_df)


In [ ]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Validation query with ALL ROWS PER MATCH
query = """
SELECT *
FROM memory.default.employees 
MATCH_RECOGNIZE (
  PARTITION BY department, region
  ORDER BY hire_date
  MEASURES 
    A.salary AS starting_salary,
    LAST(C.salary) AS ending_salary,
    MATCH_NUMBER() AS match_num
  ONE ROW PER MATCH
  AFTER MATCH SKIP PAST LAST ROW
  PATTERN (A B+ C+)
  DEFINE 
    A AS salary > 1000,
    B AS salary < 1000,
    C AS salary > 1000
);


    """
    
data = [
    {"id": 1, "name": "Alice",   "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob",     "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana",   "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
]
    
output_df = match_recognize(query, pd.DataFrame(data))
print("Match Recognize Output:")
print(output_df)


In [ ]:
query = """
SELECT * FROM memory.default.employees  MATCH_RECOGNIZE(
    PARTITION BY department, region
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        salary AS current_salary,
        RUNNING SUM(salary) AS running_sum
    ALL ROWS PER MATCH
    PATTERN (A {- B+ -} C+)
    DEFINE 
        A AS salary > 1000,
        B AS salary < 1000,
        C AS salary > 1000
);
"""

In [ ]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Example query with comprehensive CLASSIFIER usage
query = """
SELECT * FROM  memory.default.employees MATCH_RECOGNIZE(
    PARTITION BY department, region
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        CLASSIFIER(A) AS is_a_var,
        CLASSIFIER(C) AS is_c_var,
        salary AS current_salary,
        RUNNING SUM(salary) AS running_sum
    ONE ROW PER MATCH
    PATTERN (A {- B+ -} C+)
    DEFINE 
        A AS salary > 1000,
        B AS salary < 1000,
        C AS salary > 1000
);
"""

data = [
    {"id": 1, "name": "Alice", "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob",   "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana", "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
]


output_df = match_recognize(query, pd.DataFrame(data))
print("Match Recognize Output:")
print(output_df)

In [ ]:
SELECT * FROM  memory.default.employees MATCH_RECOGNIZE(
    PARTITION BY department, region
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        CLASSIFIER(A) AS is_a_var,
        CLASSIFIER(C) AS is_c_var,
        salary AS current_salary,
        RUNNING SUM(salary) AS running_sum
    ONE ROW PER MATCH
    PATTERN (A {- B+ -} C+)
    DEFINE 
        A AS salary > 1000,
        B AS salary < 1000,
        C AS salary > 1000
);

In [ ]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Example query with comprehensive CLASSIFIER usage
query = """
SELECT * FROM memory.default.employees MATCH_RECOGNIZE(
    PARTITION BY department
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ONE ROW PER MATCH
    PATTERN (^A+)
    DEFINE 
        A AS salary > 1000
);
"""

data = [
    {"id": 1, "name": "Alice", "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob",   "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana", "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
]


output_df = match_recognize(query, pd.DataFrame(data))
print("Match Recognize Output:")
print(output_df)

In [ ]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Create test data with different departments to test partition behavior
data = [
    # Sales department - First row has high salary
    {"id": 1, "name": "Alice", "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
    {"id": 2, "name": "Bob",   "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
    {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
    {"id": 4, "name": "Diana", "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
    
    # Marketing department - Last row has high salary
    {"id": 5, "name": "Eve", "department": "Marketing", "region": "East", "hire_date": "2021-01-01", "salary": 900},
    {"id": 6, "name": "Frank", "department": "Marketing", "region": "East", "hire_date": "2021-01-02", "salary": 950},
    {"id": 7, "name": "Grace", "department": "Marketing", "region": "East", "hire_date": "2021-01-03", "salary": 980},
    {"id": 8, "name": "Henry", "department": "Marketing", "region": "East", "hire_date": "2021-01-04", "salary": 1200},
    
    # IT department - All rows have high salary
    {"id": 9, "name": "Ivy", "department": "IT", "region": "North", "hire_date": "2021-01-01", "salary": 1500},
    {"id": 10, "name": "Jack", "department": "IT", "region": "North", "hire_date": "2021-01-02", "salary": 1600},
    {"id": 11, "name": "Kate", "department": "IT", "region": "North", "hire_date": "2021-01-03", "salary": 1700},
    {"id": 12, "name": "Leo", "department": "IT", "region": "North", "hire_date": "2021-01-04", "salary": 1800},
    
    # HR department - No rows have high salary
    {"id": 13, "name": "Mike", "department": "HR", "region": "South", "hire_date": "2021-01-01", "salary": 950},
    {"id": 14, "name": "Nina", "department": "HR", "region": "South", "hire_date": "2021-01-02", "salary": 980},
    {"id": 15, "name": "Oscar", "department": "HR", "region": "South", "hire_date": "2021-01-03", "salary": 990},
    {"id": 16, "name": "Pam", "department": "HR", "region": "South", "hire_date": "2021-01-04", "salary": 995},
]

df = pd.DataFrame(data)

print("Testing Pattern Anchors\n")

# Test 1: Start anchor (^) - Should match patterns starting at the beginning of a partition
query_start_anchor = """
SELECT * FROM memory.default.orders MATCH_RECOGNIZE(
    PARTITION BY department
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ONE ROW PER MATCH
    PATTERN (^A+)
    DEFINE 
        A AS salary > 1000
);
"""

print("Test 1: Start Anchor (^) - Should only match departments where first employee has salary > 1000")
output_df = match_recognize(query_start_anchor, df)
print(output_df)
print("\n")


In [ ]:

# Test 2: End anchor ($) - Should match patterns ending at the end of a partition
query_end_anchor = """
SELECT * FROM memory.default.orders MATCH_RECOGNIZE(
    PARTITION BY department
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ONE ROW PER MATCH
    PATTERN (A+$)
    DEFINE 
        A AS salary > 1000
);
"""

print("Test 2: End Anchor ($) - Should only match departments where last employee has salary > 1000")
output_df = match_recognize(query_end_anchor, df)
print(output_df)
print("\n")



In [ ]:
# Test 3: Both anchors (^$) - Should match patterns spanning the entire partition
query_both_anchors = """
SELECT * FROM memory.default.orders MATCH_RECOGNIZE(
    PARTITION BY department
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ONE ROW PER MATCH
    PATTERN (^A+$)
    DEFINE 
        A AS salary > 1000
);
"""

print("Test 3: Both Anchors (^$) - Should only match departments where ALL employees have salary > 1000")
output_df = match_recognize(query_both_anchors, df)
print(output_df)
print("\n")


In [ ]:

# Test 4: Start anchor with ALL ROWS PER MATCH to see the actual matched rows
query_start_all_rows = """
SELECT * FROM memory.default.orders MATCH_RECOGNIZE(
    PARTITION BY department
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ALL ROWS PER MATCH
    PATTERN (^A+)
    DEFINE 
        A AS salary > 1000
);
"""

print("Test 4: Start Anchor (^) with ALL ROWS PER MATCH - Shows matched rows")
output_df = match_recognize(query_start_all_rows, df)
print(output_df)


In [ ]:
# Test PERMUTE functionality
query_permute = """
SELECT * FROM memory.default.orders MATCH_RECOGNIZE(
    PARTITION BY department
    ORDER BY hire_date
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, B))
    DEFINE 
        A AS salary > 1200,
        B AS salary < 1000
);
"""

print("Test PERMUTE - Should match both orderings of A and B")
output_df = match_recognize(query_permute, df)
print(output_df)
print("\n")


In [ ]:
#next test\


In [ ]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Create test data with different permutation patterns
data = [
    # Sequence 1: Has A-B-C pattern
    {"id": 1, "seq": 1, "step": 1, "event_type": "start", "value": 100},  # A
    {"id": 2, "seq": 1, "step": 2, "event_type": "middle", "value": 200}, # B
    {"id": 3, "seq": 1, "step": 3, "event_type": "end", "value": 300},    # C
    
    # Sequence 2: Has B-A-C pattern
    {"id": 4, "seq": 2, "step": 1, "event_type": "middle", "value": 250}, # B
    {"id": 5, "seq": 2, "step": 2, "event_type": "start", "value": 150},  # A
    {"id": 6, "seq": 2, "step": 3, "event_type": "end", "value": 350},    # C
    
    # Sequence 3: Has A-C-B pattern
    {"id": 7, "seq": 3, "step": 1, "event_type": "start", "value": 175},  # A
    {"id": 8, "seq": 3, "step": 2, "event_type": "end", "value": 275},    # C
    {"id": 9, "seq": 3, "step": 3, "event_type": "middle", "value": 375}, # B
    
    # Sequence 4: Has C-B-A pattern
    {"id": 10, "seq": 4, "step": 1, "event_type": "end", "value": 225},   # C
    {"id": 11, "seq": 4, "step": 2, "event_type": "middle", "value": 325}, # B
    {"id": 12, "seq": 4, "step": 3, "event_type": "start", "value": 425},  # A
]

df = pd.DataFrame(data)

print("Testing PERMUTE Patterns\n")

# Test 1: Basic PERMUTE - Match any order of A, B, C
query_basic_permute = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        A.value AS a_value,
        B.value AS b_value,
        C.value AS c_value
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, B, C))
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 1: Basic PERMUTE - Should match all sequences with A, B, C in any order")
output_df = match_recognize(query_basic_permute, df)
print(output_df)
print("\n")

In [ ]:

# Test 4: PERMUTE with Subset Variables
query_permute_subset = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        X.value AS x_value,
        Y.value AS y_value
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, B, C))
    SUBSET
        X = (A, B),
        Y = (B, C)
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 4: PERMUTE with Subset Variables - Using subset groupings")
output_df = match_recognize(query_permute_subset, df)
print(output_df)
print("\n")


In [ ]:
# Test 2: PERMUTE with Quantifier
query_permute_quantifier = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        FIRST(A.value) AS first_a_value,
        LAST(C.value) AS last_c_value
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, B, C)+)
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 2: PERMUTE with Quantifier - Should match one or more occurrences of permutations")
output_df = match_recognize(query_permute_quantifier, df)
print(output_df)
print("\n")

In [ ]:
# Test 3: PERMUTE with ALL ROWS PER MATCH
query_permute_all_rows = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        RUNNING LAST(A.value) AS running_a_value
    ALL ROWS PER MATCH
    PATTERN (PERMUTE(A, B, C))
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 3: PERMUTE with ALL ROWS PER MATCH - Shows all matched rows")
output_df = match_recognize(query_permute_all_rows, df)
print(output_df)
print("\n")


In [ ]:

# Test 4: PERMUTE with Subset Variables
query_permute_subset = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        X.value AS x_value,
        Y.value AS y_value
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, B, C))
    SUBSET
        X = (A, B),
        Y = (B, C)
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 4: PERMUTE with Subset Variables - Using subset groupings")
output_df = match_recognize(query_permute_subset, df)
print(output_df)
print("\n")


In [ ]:
!find . -type f -name "*.py" | sort

In [ ]:

# Test 5: Nested PERMUTE patterns
query_nested_permute = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        A.value AS a_value,
        B.value AS b_value,
        C.value AS c_value
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, PERMUTE(B, C)))
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 5: Nested PERMUTE - Testing nested permutation patterns")
output_df = match_recognize(query_nested_permute, df)
print(output_df)
print("\n")

In [ ]:
# Test 6: PERMUTE with Complex Conditions
query_permute_complex = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        A.value AS start_value,
        B.value AS middle_value,
        C.value AS end_value
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, B, C))
    DEFINE 
        A AS event_type = 'start' AND A.value < NEXT(A.value),
        B AS event_type = 'middle' AND B.value > PREV(B.value),
        C AS event_type = 'end' AND C.value > FIRST(A.value)
);
"""

print("Test 6: PERMUTE with Complex Conditions - Testing complex pattern definitions")
output_df = match_recognize(query_permute_complex, df)
print(output_df)
print("\n")

In [ ]:








# Test 7: PERMUTE with Edge Cases
query_permute_edge_cases = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        A.value AS a_value,
        LAST(B.value) AS last_b_value,
        FIRST(C.value) AS first_c_value
    ALL ROWS PER MATCH
    PATTERN (PERMUTE(A, B?, C?))
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

print("Test 7: PERMUTE with Edge Cases - Testing optional elements")
output_df = match_recognize(query_permute_edge_cases, df)
print(output_df)


In [8]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Create test data for PERMUTE with subset variables
data = [
    # Sequence 1: Has A-B-C pattern
    {"id": 1, "seq": 1, "step": 1, "event_type": "start", "value": 100},  # A
    {"id": 2, "seq": 1, "step": 2, "event_type": "middle", "value": 200}, # B
    {"id": 3, "seq": 1, "step": 3, "event_type": "end", "value": 300},    # C
    
    # Sequence 2: Has B-A-C pattern
    {"id": 4, "seq": 2, "step": 1, "event_type": "middle", "value": 250}, # B
    {"id": 5, "seq": 2, "step": 2, "event_type": "start", "value": 150},  # A
    {"id": 6, "seq": 2, "step": 3, "event_type": "end", "value": 350},    # C
    
    # Sequence 3: Has A-C-B pattern
    {"id": 7, "seq": 3, "step": 1, "event_type": "start", "value": 175},  # A
    {"id": 8, "seq": 3, "step": 2, "event_type": "end", "value": 275},    # C
    {"id": 9, "seq": 3, "step": 3, "event_type": "middle", "value": 375}, # B
    
    # Sequence 4: Has C-B-A pattern
    {"id": 10, "seq": 4, "step": 1, "event_type": "end", "value": 225},   # C
    {"id": 11, "seq": 4, "step": 2, "event_type": "middle", "value": 325}, # B
    {"id": 12, "seq": 4, "step": 3, "event_type": "start", "value": 425},  # A
]

df = pd.DataFrame(data)

print("Testing PERMUTE with Subset Variables - Trino Compatibility\n")

# Test: PERMUTE with Subset Variables and Updated Processing Logic
query_permute_subset = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        A.value AS start_value,
        B.value AS middle_value,
        C.value AS end_value
    ONE ROW PER MATCH
    PATTERN (PERMUTE(A, B, C))
    DEFINE 
        A AS event_type = 'start' AND A.value < NEXT(A.value),
        B AS event_type = 'middle' AND B.value > PREV(B.value),
        C AS event_type = 'end' AND C.value > FIRST(A.value)
);
"""

print("PERMUTE with Subset Variables - Using last component variable that appears in match:")
output_df = match_recognize(query_permute_subset, df)
print(output_df)


DEBUG:src.parser.match_recognize_extractor:Full statement text: SELECT * FROM memory.default.op2 MATCH_RECOGNIZE( PARTITION BY seq ORDER BY step MEASURES MATCH_NUMBER() AS match_num ONE ROW PER MATCH PATTERN (()) DEFINE A AS event_type = 'start' -- DEFINE is ignored since pattern is empty );
DEBUG:src.parser.match_recognize_extractor:Extracted SELECT clause: SelectClause(items=[SelectItem(expression=*, metadata={})])
DEBUG:src.parser.match_recognize_extractor:Extracted FROM clause: FromClause(table='memory')
DEBUG:src.parser.match_recognize_extractor:Visiting PatternRecognition context
DEBUG:src.parser.match_recognize_extractor:Extracted PARTITION BY: PartitionByClause(columns=['seq'])
DEBUG:src.parser.match_recognize_extractor:Extracted ORDER BY: OrderByClause(sort_items=[SortItem(column='step', ordering='ASC', nulls_ordering=None)])
DEBUG:src.parser.match_recognize_extractor:Extracted MEASURES: MeasuresClause(measures=[Measure(expression='MATCH_NUMBER()', alias='match_num', metadata=

Testing PERMUTE with Subset Variables - Trino Compatibility

PERMUTE with Subset Variables - Using last component variable that appears in match:
Pattern value: '()'
Empty pattern detected - allowing any variable definition
Raw rows_per_match text: ONEROWPERMATCH
Using rows_per_match mode: RowsPerMatch.ONE_ROW
Measure semantics: {'match_num': 'FINAL'}
Match config: rows_per_match=RowsPerMatch.ONE_ROW, all_rows=False
Start state is accepting - pattern allows empty matches
DFA state 0 has transitions for variables: []
Built DFA with 1 states
  State 0: Accept, Variables: None, Excluded: None, Transitions: 
Find matches with all_rows=False, show_empty=True, include_unmatched=False
Starting match at index 0, state: State 0 (Accept, Vars: None)
Found potential empty match at index 0 - start state is accepting
Testing row 0, data: {'id': 1, 'seq': 1, 'step': 1, 'event_type': 'start', 'value': 100}
No valid transition from accepting state 0 at row 0
No valid transition from state 0 at row 0
U

KeyError: 'pattern_var'

In [ ]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Test for SUBSET clause and union variables
print("Test: SUBSET clause with union variables")
query_subset = """
SELECT * FROM memory.default.orders MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        FIRST(X.value) AS first_x_value,
        LAST(X.value) AS last_x_value,
        CLASSIFIER(X) AS x_classifier
    ONE ROW PER MATCH
    PATTERN (A B C)
    SUBSET X = (A, C)
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

output_df = match_recognize(query_subset, df)
print(output_df)
print("\n")

In [ ]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Test for empty pattern matching
print("Test: Empty pattern matching with ()")
query_empty_pattern = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        MATCH_NUMBER() AS match_num
    ONE ROW PER MATCH
    PATTERN (())
    DEFINE
        A AS event_type = 'start'  -- DEFINE is ignored since pattern is empty
);
"""

output_df = match_recognize(query_empty_pattern, df)
print(output_df)
print("\n")

In [ ]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Test for RUNNING vs FINAL semantics
print("Test: RUNNING vs FINAL semantics")
query_semantics = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        RUNNING SUM(value) AS running_sum,
        FINAL SUM(value) AS final_sum,
        MATCH_NUMBER() AS match_num
    ALL ROWS PER MATCH
    PATTERN (A B C)
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

output_df = match_recognize(query_semantics, df)
print(output_df)
print("\n")

In [ ]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Test for ALL ROWS PER MATCH WITH UNMATCHED ROWS
print("Test: ALL ROWS PER MATCH WITH UNMATCHED ROWS")
query_unmatched = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ALL ROWS PER MATCH WITH UNMATCHED ROWS
    PATTERN (A B)
    DEFINE 
        A AS event_type = 'start' AND step = 1,
        B AS event_type = 'middle' AND step = 2
);
"""

output_df = match_recognize(query_unmatched, df)
print(output_df)
print("\n")

In [ ]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Test for Pattern Exclusion {- ... -}
print("Test: Pattern Exclusion Syntax")
query_exclusion = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ALL ROWS PER MATCH
    PATTERN (A {- B -} C)
    DEFINE 
        A AS event_type = 'start',
        B AS event_type = 'middle',
        C AS event_type = 'end'
);
"""

output_df = match_recognize(query_exclusion, df)
print(output_df)
print("\n")

In [ ]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Test for reluctant quantifiers
print("Test: Reluctant Quantifiers")
query_reluctant = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num,
        COUNT(A.*) AS a_count,
        COUNT(B.*) AS b_count
    ONE ROW PER MATCH
    PATTERN (A+? B+)
    DEFINE 
        A AS step <= 2,
        B AS step > 1
);
"""

output_df = match_recognize(query_reluctant, df)
print(output_df)
print("\n")

In [ ]:
import pandas as pd
from src.executor.match_recognize import match_recognize

# Test for AFTER MATCH SKIP options
print("Test: AFTER MATCH SKIP options")
query_skip = """
SELECT * FROM memory.default.op2 MATCH_RECOGNIZE(
    PARTITION BY seq
    ORDER BY step
    MEASURES 
        CLASSIFIER() AS pattern_var,
        MATCH_NUMBER() AS match_num
    ONE ROW PER MATCH
    AFTER MATCH SKIP TO NEXT ROW
    PATTERN (A B+)
    DEFINE 
        A AS step = 1,
        B AS step > 1
);
"""

output_df = match_recognize(query_skip, df)
print(output_df)
print("\n")